In [1]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/{accelerator}/torch_nightly.html
  
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.backends.cudnn.enabled)

!pip install fastai

import fastai
from fastai import *
from fastai.vision import *

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
    100% |████████████████████████████████| 578.2MB 27kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x60c8e000 @  0x7fed78e272a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070
1.0.0.dev20181029
True
True
    100% |████████████████████████████████| 112kB 5.9MB/s 
    100% |████████████████████████████████| 9.0MB 4.4MB/s 
    100% |████████████████████████████████| 61kB 22.0MB/s 
    100% |████████████████████████████████| 1.0MB 18.6MB/s 
    100% |████████████████████████████████| 112kB 27.7MB/s 
    100% |████████████████████████████████| 112kB 29.9MB/s 
    100% |████████████████████████████████| 2.2MB 12.0MB/s 
  Running setup.py bdist_wheel for prometheus-client ... - \ done
 

In [0]:
#install/import pytorch
#from os.path import exists
#from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
#platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
#cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
#accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

#!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
#import torch

In [0]:
#install/import fastai
#!pip install fastai
#!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
#import cv2
#from os import path
#from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
#platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())##

#accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

#!pip install pillow==4.1.1


#!pip install image
#from fastai.imports import *

In [0]:
from fastai.vision import *
from fastai import *

import numpy as np
import pandas as pd

In [13]:
!ls -alh

total 90M
drwxr-xr-x 1 root root 4.0K Oct 31 03:51 .
drwxr-xr-x 1 root root 4.0K Oct 31 03:50 ..
drwxr-xr-x 4 root root 4.0K Oct 29 16:30 .config
drwxr-xr-x 2 root root 4.0K Oct 29 16:42 sample_data
-rw-r--r-- 1 root root  89M Oct 31 03:53 USA_SNAKES.zip


In [14]:
!unzip USA_SNAKES.zip

Archive:  USA_SNAKES.zip
   creating: USA_SNAKES/
   creating: USA_SNAKES/Coral/
  inflating: USA_SNAKES/Coral/1. 15390754139_6b85681f64_b.jpg  
  inflating: USA_SNAKES/Coral/1. 220px-micruroides_euryxanthus.jpg  
 extracting: USA_SNAKES/Coral/1. 2562524532_6c6a35ebbe_b.jpg  
 extracting: USA_SNAKES/Coral/1. coral-snake.png  
  inflating: USA_SNAKES/Coral/1. eastern-coral-snake-660x400.jpg  
  inflating: USA_SNAKES/Coral/1. hqdefault.jpg  
  inflating: USA_SNAKES/Coral/1. micrurus_tener.jpg  
  inflating: USA_SNAKES/Coral/10. coral02.jpg  
  inflating: USA_SNAKES/Coral/10. micruroides-euryxanthus3a-rancho-viejo-s-la-purica-b51-623x1024.jpg  
  inflating: USA_SNAKES/Coral/10. texas-coral-snake-1.jpg  
  inflating: USA_SNAKES/Coral/100. 171323533-texas-coral-snake-otter-fallen-leaves-forest-soil.jpg  
  inflating: USA_SNAKES/Coral/100. head-and-nek-markings-arizona-coral-snake-gary-nafis.jpg  
  inflating: USA_SNAKES/Coral/11. 6610312261_fd53de36fd.jpg  
  inflating: USA_SNAKES/Coral/11.

In [0]:
!mkdir USA_SNAKES/train

In [0]:
!mv USA_SNAKES/PitViper USA_SNAKES/train
!mv USA_SNAKES/Mild USA_SNAKES/train
!mv USA_SNAKES/Harmless USA_SNAKES/train
!mv USA_SNAKES/Coral USA_SNAKES/train

In [0]:
# Thanks https://stackoverflow.com/questions/48547660/attributeerror-module-pil-image-has-no-attribute-register-extensions
# workaround 
#from PIL import Image
#def register_extension(id, extension): Image.EXTENSION[extension.lower()] = id.upper()
#Image.register_extension = register_extension
#def register_extensions(id, extensions): 
#  for extension in extensions: register_extension(id, extension)
#Image.register_extensions = register_extensions

In [0]:
#!pip install PIL
#!pip install image

In [0]:
tfms=get_transforms(max_zoom=1.1)

In [18]:
  data = ImageDataBunch.from_folder("USA_SNAKES/", ds_tfms=tfms, bs=28, size=224, valid_pct=0.2, num_workers=0)
data.normalize(imagenet_stats)

In [0]:
#!pip install piexif

In [0]:
#import piexif

In [0]:
#from pathlib import Path

In [0]:
#pathlist = Path("USA_SNAKES").glob('**/*.asm')
#for path in pathlist:
#    # because path is object not string
#    path_in_str = str(path)
#    piexif.remove(path_in_str)

In [22]:
learn = create_cnn(data, models.resnet34, metrics=accuracy)
learn.fit_one_cycle(5, 0.01)

Total time: 13:18
epoch  train_loss  valid_loss  accuracy
1      0.849770    0.697756    0.745167  (02:39)
2      0.739596    0.842652    0.697715  (02:41)
3      0.658813    0.596348    0.757469  (02:42)
4      0.514135    0.437123    0.836555  (02:41)
5      0.396697    0.437297    0.838313  (02:32)



In [23]:
accuracy(*learn.get_preds())


tensor(0.8383)